In [28]:
#!pip install torch
import sqlite3
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import statistics
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split

In [4]:
# Создаем соединение с базой данных
conn = sqlite3.connect('garpix_db_0.0.2.db')

In [12]:
# Формируем DataFrame  
df = pd.read_sql_query("""
    SELECT
       calculation_id,
       size_width, 
       size_height, 
       size_length
    FROM boxes
""", conn)
print(df.head())

   calculation_id  size_width  size_height  size_length
0           16900        98.0        143.0        385.0
1           16900       196.0        198.0        363.0
2           16900       277.0        190.0        407.0
3           16900       170.0        325.0        255.0
4           16900       250.0        350.0        160.0


In [22]:
grouped_df = df.groupby('calculation_id').agg(list)
grouped_df.groupby('calculation_id').agg(pd.Series.tolist)
grouped_df = df.groupby('calculation_id')[['size_width', 'size_height', 'size_length']].agg(list)
temp_index = [index for index in grouped_df['size_width'].index]
def get_data_index_1(index):
	temp_1 = []
	len_temp = len(grouped_df['size_width'][index])
	for len_x in range(len_temp):
		temp_1 = temp_1 + [grouped_df['size_width'][index][len_x], grouped_df['size_height'][index][len_x], grouped_df['size_length'][index][len_x]]
	return temp_1

data_x =  [torch.tensor(get_data_index_1(index)) for index in temp_index]
for x in range(0, len(data_x)):
	data_x[x] = data_x[x].to(torch.float)
	data_x[x] = data_x[x] / data_x[x].max()
data_y = pad_sequence(data_x, batch_first=True)

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=8, batch_first=True)
        self.fc = nn.Linear(8, 1)
    def forward(self, x):
        output, _ = self.rnn(x)
        x = self.fc(output[:, -1, :])
        return x.squeeze()

In [27]:
model = Net()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
model.to(device)

num_samples = 1000
data = []
for i in range(len(data_x)):
    tmp = torch.from_numpy(data_y[i].numpy()).reshape(8547,1)
    data.append(tmp.float().to(device),)
data_tensor = pad_sequence(data, batch_first=True)
data_tensor.to(device)

tensor([[[0.4925],
         [0.7814],
         [0.8141],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.7500],
         [0.6250],
         [1.0000],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.5161],
         [1.0000],
         [0.7742],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        ...,

        [[1.0000],
         [1.0000],
         [1.0000],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[1.0000],
         [1.0000],
         [1.0000],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[1.0000],
         [1.0000],
         [1.0000],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]]], device='cuda:0')

In [ ]:

model = torch.load('model_result_5.pt')